In [1]:
from eeggan.data.create_dataset import load_dataset
from eeggan.data.create_dataset import ZCA_whitening
from moabb.datasets import Schirrmeister2017
from braindecode.preprocessing import exponential_moving_standardize
import mne
from typing import List, Tuple
import numpy as np
from braindecode.datasets import create_from_X_y


In [2]:
CHANNELS = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4',
            'P8', 'O1', 'O2', 'M1', 'M2']                                                                               

FS = 256

SEGMENT_IVAL = (-0.5, 2.5)
INPUT_LENGTH = int((SEGMENT_IVAL[1] - SEGMENT_IVAL[0]) * FS)
CLASSDICT = {'right_hand':0, 'rest': 1,}



In [3]:
from braindecode.models.deep4 import Deep4Net
import torch

cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmarch = True

n_classes = 2
n_chans = len(CHANNELS)
input_window_samples = X.shape[-1]

NameError: name 'X' is not defined

In [12]:
model = Deep4Net(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto'
)

if cuda:
    model.cuda()

In [19]:
windows_dataset.info

AttributeError: 'BaseConcatDataset' object has no attribute 'info'

In [16]:
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier

lr = 1 * 0.01
weight_decay = 0.5 * 0.001
batch_size = 64
n_epochs = 4


clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)

In [ ]:
clf.fit(train_set, y=None, epochs=n_epochs)

In [4]:
from braindecode.datasets import MOABBDataset
dataset = MOABBDataset(dataset_name="Schirrmeister2017", subject_ids=[2])
dataset.description


Extracting EDF parameters from /home/samuel/mne_data/MNE-schirrmeister2017-data/robintibor/high-gamma-dataset/raw/master/data/train/2.edf...
EDF file detected
Channel 'EEG Fp1' recognized as type EEG (renamed to 'Fp1').
Channel 'EEG Fp2' recognized as type EEG (renamed to 'Fp2').
Channel 'EEG Fpz' recognized as type EEG (renamed to 'Fpz').
Channel 'EEG F7' recognized as type EEG (renamed to 'F7').
Channel 'EEG F3' recognized as type EEG (renamed to 'F3').
Channel 'EEG Fz' recognized as type EEG (renamed to 'Fz').
Channel 'EEG F4' recognized as type EEG (renamed to 'F4').
Channel 'EEG F8' recognized as type EEG (renamed to 'F8').
Channel 'EEG FC5' recognized as type EEG (renamed to 'FC5').
Channel 'EEG FC1' recognized as type EEG (renamed to 'FC1').
Channel 'EEG FC2' recognized as type EEG (renamed to 'FC2').
Channel 'EEG FC6' recognized as type EEG (renamed to 'FC6').
Channel 'EEG M1' recognized as type EEG (renamed to 'M1').
Channel 'EEG T7' recognized as type EEG (renamed to 'T7').
C

,subject,session,run
0,2,session_0,train
1,2,session_0,test


In [5]:
from braindecode.preprocessing import (
    exponential_moving_standardize, preprocess, Preprocessor)

# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000
# Factor to convert from V to uV
factor = 1e6

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
                 factor_new=factor_new, init_block_size=init_block_size)
]

# Transform the data
preprocess(dataset, preprocessors)

Reading 0 ... 3131499  =      0.000 ...  6262.998 secs...


: 

: 